<a href="https://colab.research.google.com/github/MinnThikeTunn/tinyModel-philosopherModel-/blob/main/Philosophy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Connect to a GPU runtime (T4 or better)
!nvidia-smi

# Install required packages
!pip install -q transformers datasets accelerate peft bitsandbytes torch sentencepiece

# For logging (optional)
!pip install -q wandb

Fri Apr 11 04:38:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

pip install gcsfs fsspec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the philosophy dialogue dataset
dataset = load_dataset("Hypersniper/philosophy_dialogue")  # Note: Corrected dataset name if needed

# Inspect the dataset
print(dataset)
print("\nSample dialogue:")
print(dataset["train"][0])
print(dataset)       # PEFT LoRA adapters :contentReference[oaicite:5]{index=5}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

database_main_socrates.json:   0%|          | 0.00/595k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/395 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 395
    })
})

Sample dialogue:
{'instruction': 'What form of art resonates with you the most?', 'output': 'Ah, a fascinating question indeed! As Socrates, I must first clarify that my perception of art may differ from the common view. For me, art is not merely a form of aesthetic expression but a vehicle for truth and virtue. Art, in its highest form, should elevate the soul and stimulate thought, not just please the eyes.\n\nIf I were to choose a form of art that resonates with me the most, it would be the art of dialogue, the art of dialectic. This art, which I have practiced throughout my life, is a form of intellectual wrestling, where ideas are scrutinized, assumptions are challenged, and wisdom is pursued. It is through this art that we can reach a higher understanding of ourselves and the world.\n\nHowever, I also appreciate the art of sculpture, as it represents the ideal of physi

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for OPT
model_name = "facebook/opt-2.7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

def preprocess_function(example):  # Changed from examples to example (singular)
    # Create the instruction-output template
    full_text = f"Instruction: {example['instruction']}\nOutput: {example['output']}"

    # Tokenize the instruction part separately to find where output begins
    instruction_text = f"Instruction: {example['instruction']}\nOutput: "
    instruction_tokens = tokenizer(instruction_text, add_special_tokens=False)

    # Tokenize the output part (with EOS token)
    output_tokens = tokenizer(example['output'], add_special_tokens=True)

    # Combine tokens
    input_ids = instruction_tokens["input_ids"] + output_tokens["input_ids"]

    # Create labels (-100 for instruction part, actual tokens for output)
    labels = [-100] * len(instruction_tokens["input_ids"]) + output_tokens["input_ids"]

    # Create attention mask
    attention_mask = [1] * len(input_ids)

    # Pad or truncate to max_length (512)
    if len(input_ids) < 512:
        # Pad
        pad_length = 512 - len(input_ids)
        input_ids = input_ids + [tokenizer.pad_token_id] * pad_length
        labels = labels + [-100] * pad_length
        attention_mask = attention_mask + [0] * pad_length
    else:
        # Truncate
        input_ids = input_ids[:512]
        labels = labels[:512]
        attention_mask = attention_mask[:512]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# Apply preprocessing
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=False,  # Process one example at a time
    remove_columns=dataset["train"].column_names
)

# Split into train and validation
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

In [ ]:
# First, install the correct versions of all required packages
!pip install -q torch torchvision torchaudio
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/accelerate.git
!pip install -q datasets bitsandbytes sentencepiece

# For CUDA 11.8 (Colab's default as of 2024)
!pip install --quiet bitsandbytes
!pip install --quiet --upgrade transformers # Install latest version of transformers
!pip install --quiet --upgrade accelerate
!pip install --quiet sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
!pip install transformers bitsandbytes accelerate

In [ ]:
# Note, if you don't want to reinstall BNBs dependencies, append the `--no-deps` flag!
!pip install --force-reinstall 'https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/continuous-release_multi-backend-refactor/bitsandbytes-0.44.1.dev0-py3-none-macosx_13_1_arm64.whl'

ERROR: bitsandbytes-0.44.1.dev0-py3-none-macosx_13_1_arm64.whl is not a supported wheel on this platform.


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

pytorch_model.bin:   0%|          | 0.00/5.30G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]  # For OPT model
)

# Prepare model for PEFT
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 5,242,880 || all params: 2,656,839,680 || trainable%: 0.1973


In [ ]:
# Save the trained model
model.save_pretrained("./opt-2.7b-philosophy-lora")

# Save tokenizer
tokenizer.save_pretrained("./opt-2.7b-philosophy-lora")

('./opt-2.7b-philosophy-lora/tokenizer_config.json',
 './opt-2.7b-philosophy-lora/special_tokens_map.json',
 './opt-2.7b-philosophy-lora/vocab.json',
 './opt-2.7b-philosophy-lora/merges.txt',
 './opt-2.7b-philosophy-lora/added_tokens.json',
 './opt-2.7b-philosophy-lora/tokenizer.json')

In [ ]:

from transformers import pipeline

# Create text generation pipeline
philosophy_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

# Test the model
prompt = "Instruction: what is Wiston Charchey?\nOutput:"
result = philosophy_pipe(prompt)
print(result[0]['generated_text'])

Device set to use cuda:0


Instruction: what is Wiston Charchey?
Output: what is Wiston Charchey?

The first and the only time that I heard the name Wiston Charchey was when I was in the first grade. The class was learning about the famous African explorer, Lewis and Clark. I don’t recall who told us about Charchey, but I do remember the name.

I was always intrigued by the name, so I looked it up. Turns out, it’s a real place, in the United States. It’s located in the far north of Michigan, and is also known as “the land of the polar bears”.

I remember wondering what was so special about this place. I assumed it was a really remote place, but it turns out that it’s only about 200 miles from Canada.

It was in the middle of nowhere, but I still remember that I had heard about this place. I didn’t know anything about the people who lived there, or what they did.

I never had the opportunity to visit the place, and it’s not as if I’ve been there many times since.

Today, I’m going to share with you what I know ab

In [ ]:
from huggingface_hub import notebook_login

notebook_login()  # Follow the auth steps

model.push_to_hub("mari-a-12a/opt-2.7b-philosophy")
tokenizer.push_to_hub("mari-a-12a/opt-2.7b-philosophy")

adapter_model.safetensors:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mari-a-12a/opt-2.7b-philosophy/commit/2efb0e9d320046acf5ae56f03fd5d60ac4717165', commit_message='Upload tokenizer', commit_description='', oid='2efb0e9d320046acf5ae56f03fd5d60ac4717165', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mari-a-12a/opt-2.7b-philosophy', endpoint='https://huggingface.co', repo_type='model', repo_id='mari-a-12a/opt-2.7b-philosophy'), pr_revision=None, pr_num=None)

In [ ]:
!pip install gradio transformers
import gradio as gr
from transformers import pipeline

philo_pipe = pipeline("text-generation", model="mari-a-12a/opt-2.7b-philosophy")

def respond(message):
    full_prompt = f"Instruction: {message}\nOutput:"
    output = philo_pipe(full_prompt, max_length=100)[0]['generated_text']
    return output.split("Output:")[1].strip()

gr.Interface(
    fn=respond,
    inputs=gr.Textbox(lines=2, placeholder="Ask your philosophical question..."),
    outputs="text"
).launch(share=True)  # Creates a public link

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a040bea72aa149fe8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
